In [56]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
from selenium import webdriver
import time
import re
from selenium.webdriver.common.keys import Keys 
from urllib.parse import quote_plus
from urllib.request import urlopen
import os

In [57]:
import webbrowser
def Driver():
    try:
        path = chromedriver_autoinstaller.install()
        
    except FileNotFoundError as err:
        print("크롬 브라우저를 찾을 수 없습니다. 설치 후 재시도 하시기 바랍니다.")
        webbrowser.open("https://www.google.com/intl/ko/chrome/")
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    #options.add_argument("headless")
    options.add_argument('--start-fullscreen')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    wd = webdriver.Chrome(options = options)
    return wd

In [58]:
# 베이스 URL과 상업적 이용 가능 옵션
def make_url(word):
    base_url = 'https://unsplash.com/ko/s/%EC%82%AC%EC%A7%84/'
    return base_url + word

selenium과 bs4로 element로 이미지의 url를 추출하려했으나, 웹사이트 보안 정책인지 뭔지는 몰라도 element를 찾을 수가 없어서 추출이 불가 했다.
그런 이유로 unplash에서 권장하는 정책으로 Rest API 요쳥을 통해서 image URL을 추출했다. 더욱 빠르기도하고 방법도 간단했다.
다만 demo버전의 권한으로는 시간당 50번의 요청밖에 하지 못하고, 페이지당 최대 30개의 이미지를 추출할 수 있어서 최대 50 x 30 시간 당 최대 1500개 까지만 크롤링이 가능하다는 단점이 있다.

In [136]:
import requests
search_word = 'asian'
images = []


# images['results'][0]['urls']['regular']

45번째 페이지에서 요청이 거부되었습니다. 45번째 페이지 부터 다시 시작해주세요! Expecting value: line 1 column 1 (char 0)


In [129]:
len(images)
        

1319

In [130]:
# 해당하는 폴더가 없을 경우 생성해주는 함수
def makedirs(path): 
   try: 
        os.makedirs(path) 
   except OSError: 
       if not os.path.isdir(path): 
           raise

In [ ]:
class crawler:
    def __init__(self, query, start_page=1):
        self.query = query
        self.start_page = start_page
        self.images = []
    
    def activate(self):
        
        res = requests.get(f'https://api.unsplash.com/search/photos/?client_id=your_access_key&query={search_word}&page={i}&per_page=30')
        response = res.json()
        total_page = response['total_pages']
        while self.start_page <= total_page:
            for i in range(self.start_page,total_page+1):
                try:                                      
                    for element in response['results']:
                        self.images.append(element['urls']['regular'])   
                except Exception as e:
                    print(f"{i}번째 페이지에서 요청이 거부되었습니다. 한 시간 뒤에 {i}번째 페이지 부터 다시 시작 합니다!", e)
                    break
                self.start_page = i
                time.sleep(60*60+1)       
        print(f'추출한 image url의 개수는 {len(self.images)}입니다.')
    
    def execute_images(self):
        def save_images(image_url, paths, i):
            import base64
            import os
                
            if 'data:' in str(image_url):
                pass
            else:
                try:
                    t= urlopen(image_url).read()
                    file = open(os.path.join(paths, str(i)+".gif"), 'wb')
                    file.write(t)
                except Exception as e:
                    print(e)

        for i, image in enumerate(self.images,1):
            base_path = os.path.abspath(os.getcwd())
            save_path = base_path + f'/{self.query}'
            makedirs(save_path)
            save_images(image, save_path, str(i), i)

In [142]:
# 현재 디렉토리 위치
import os



'c:\\Users\\mgd81\\github\\Crawlers_py'